# Setup

Required libraries are loaded, all task psms will be merged into 1 TSV file and will be read in a dataframe.

In [1]:
import pandas as pd
from utils.preprocess import *

In [24]:
merge_psm_files('../../massivekb_dataset/working_results/results_calibrated', '../../massivekb_dataset/working_results/results_calibrated_merged.tsv',['filename','level 1','scan','RT','sequence','task_id','iRT'])

All TSV files have been merged into ../../massivekb_dataset/working_results/results_calibrated_merged.tsv


In [32]:
column_names = ["filename", "level 1", "scan", "RT", "sequence", "task_id", "iRT"]
df = pd.read_csv('../../massivekb_dataset/working_results/results_calibrated_merged.tsv', sep='\t', names=column_names, index_col=False)

ParserError: Error tokenizing data. C error: Expected 7 fields in line 2, saw 8


In [31]:
df.head(10)

,filename,level 1,scan,RT,sequence,task_id,iRT
0,PTSS4201SN16441.mzML,30,20919,58.286275,PVTVVVLK,04fe276206f54b3c8f81798ea92aa6e3,NaN
1,PTSS4201SN16441.mzML,255,16551,50.065410,SASSSAAGSPGGLTSLQQQK,04fe276206f54b3c8f81798ea92aa6e3,NaN
2,PTSS4201SN16441.mzML,275,28064,71.616320,LRNDLLNIIR,04fe276206f54b3c8f81798ea92aa6e3,NaN
3,PTSS4201SN16441.mzML,299,17303,51.487835,MGPSRLVRGPR,04fe276206f54b3c8f81798ea92aa6e3,NaN
4,PTSS4201SN16441.mzML,307,44559,103.201400,LQPMLLEMER,04fe276206f54b3c8f81798ea92aa6e3,NaN
5,PTSS4201SN16441.mzML,320,19567,55.739282,NIIIAGPPGTGK,04fe276206f54b3c8f81798ea92aa6e3,NaN
6,PTSS4201SN16441.mzML,337,44613,103.309380,QSVFAILR,04fe276206f54b3c8f81798ea92aa6e3,NaN
7,PTSS4201SN16441.mzML,380,44684,103.443740,QVVNIPSFIVR,04fe276206f54b3c8f81798ea92aa6e3,NaN
8,PTSS4201SN16441.mzML,427,15588,48.283515,IFIHGHKVQTKR,04fe276206f54b3c8f81798ea92aa6e3,NaN
9,PTSS4201SN16441.mzML,510,22891,61.913960,LPGIPIK,04fe276206f54b3c8f81798ea92aa6e3,NaN


In [26]:
df = df.drop('level 1', axis=1)

# Base information dataframe

This section will show the basic properties of the dataset to get an idea of what the dataset looks like.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25915444 entries, 0 to 25915443
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   filename  object 
 1   scan      int64  
 2   RT        float64
 3   sequence  object 
 4   task_id   object 
 5   iRT       float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.2+ GB


In [28]:
df.describe()

,scan,RT,iRT
count,2.591544e+07,2.591544e+07,0.0
mean,4.701977e+04,1.131254e+02,NaN
std,3.171750e+04,6.800618e+01,NaN
min,2.000000e+00,8.289219e-03,NaN
25%,2.108300e+04,5.538618e+01,NaN
50%,4.146700e+04,1.033883e+02,NaN
75%,6.772900e+04,1.640497e+02,NaN
max,2.458620e+05,6.000026e+02,NaN


In [29]:
df.head(10)

,filename,scan,RT,sequence,task_id,iRT
0,PTSS4201SN16441.mzML,20919,58.286275,PVTVVVLK,04fe276206f54b3c8f81798ea92aa6e3,NaN
1,PTSS4201SN16441.mzML,16551,50.065410,SASSSAAGSPGGLTSLQQQK,04fe276206f54b3c8f81798ea92aa6e3,NaN
2,PTSS4201SN16441.mzML,28064,71.616320,LRNDLLNIIR,04fe276206f54b3c8f81798ea92aa6e3,NaN
3,PTSS4201SN16441.mzML,17303,51.487835,MGPSRLVRGPR,04fe276206f54b3c8f81798ea92aa6e3,NaN
4,PTSS4201SN16441.mzML,44559,103.201400,LQPMLLEMER,04fe276206f54b3c8f81798ea92aa6e3,NaN
5,PTSS4201SN16441.mzML,19567,55.739282,NIIIAGPPGTGK,04fe276206f54b3c8f81798ea92aa6e3,NaN
6,PTSS4201SN16441.mzML,44613,103.309380,QSVFAILR,04fe276206f54b3c8f81798ea92aa6e3,NaN
7,PTSS4201SN16441.mzML,44684,103.443740,QVVNIPSFIVR,04fe276206f54b3c8f81798ea92aa6e3,NaN
8,PTSS4201SN16441.mzML,15588,48.283515,IFIHGHKVQTKR,04fe276206f54b3c8f81798ea92aa6e3,NaN
9,PTSS4201SN16441.mzML,22891,61.913960,LPGIPIK,04fe276206f54b3c8f81798ea92aa6e3,NaN
